In [3]:
import joblib
import json

# Load the scaler
scaler = joblib.load('scaler.pkl')
scaler_data = {'mean': scaler.mean_.tolist(), 'scale': scaler.scale_.tolist()}
    
# Save scaler data to a JSON file
with open('scaler.json', 'w') as f:
    json.dump(scaler_data, f)


In [5]:
import tensorflow as tf
import json

class CustomModel(tf.Module):
    def __init__(self, model, scaler_data):
        self.model = model
        self.mean = tf.constant(scaler_data['mean'], dtype=tf.float32)
        self.scale = tf.constant(scaler_data['scale'], dtype=tf.float32)

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 126], dtype=tf.float32)])
    def __call__(self, inputs):
        # Preprocess the inputs
        scaled_inputs = (inputs - self.mean) / self.scale
        # Pass through the model
        return self.model(scaled_inputs)

# Load the scaler data
with open('scaler.json', 'r') as f:
    scaler_data = json.load(f)

# Load the trained model
model = tf.keras.models.load_model('./model-2024-06-21.h5')

# Create the custom model
custom_model = CustomModel(model, scaler_data)

# Convert the custom model to TFLite
converter = tf.lite.TFLiteConverter.from_concrete_functions([custom_model.__call__.get_concrete_function()])
tflite_model = converter.convert()

# Save the TFLite model
with open('model_with_preprocessing.tflite', 'wb') as f:
    f.write(tflite_model)
